In [2]:
from graphene import ObjectType, String, Int, List, Schema, Mutation, Field

# データベースの代わりとなるリスト
books = [
    {'id': 1, 'title': 'Pythonプログラミング', 'author': '山田太郎'},
    {'id': 2, 'title': 'GraphQL入門', 'author': '佐藤花子'}
]

class Book(ObjectType):
    id = Int()
    title = String()
    author = String()

class Query(ObjectType):
    books = List(Book)
    book = Field(Book, id=Int(required=True))

    def resolve_books(self, info):
        return books

    def resolve_book(self, info, id):
        for book in books:
            if book['id'] == id:
                return book
        return None

schema = Schema(query=Query)

In [3]:
# 全ての書籍を取得
query_string = '''
{
    books {
        id
        title
        author
    }
}
'''

result = schema.execute(query_string)
print(result.data)

{'books': [{'id': 1, 'title': 'Pythonプログラミング', 'author': '山田太郎'}, {'id': 2, 'title': 'GraphQL入門', 'author': '佐藤花子'}]}


In [4]:
# 特定の書籍を取得
single_query_string = '''
{
    book(id: 2) {
        title
        author
    }
}
'''

result = schema.execute(single_query_string)
print(result.data)

{'book': {'title': 'GraphQL入門', 'author': '佐藤花子'}}


In [5]:
class CreateBook(Mutation):
    class Arguments:
        title = String(required=True)
        author = String(required=True)

    book = Field(Book)

    def mutate(self, info, title, author):
        new_id = max(book['id'] for book in books) + 1
        new_book = {'id': new_id, 'title': title, 'author': author}
        books.append(new_book)
        return CreateBook(book=new_book)
    
class Mutations(ObjectType):
    create_book = CreateBook.Field()

schema = Schema(query=Query, mutation=Mutations)

In [6]:
mutation_string = '''
mutation {
    createBook(title: "GraphQL実践", author: "鈴木一郎"){
        book {
            id
            title
            author
        }
    }
}
'''

result = schema.execute(mutation_string)
print(result.data)

{'createBook': {'book': {'id': 3, 'title': 'GraphQL実践', 'author': '鈴木一郎'}}}
